# Experimento 5 

Hiperparmetro: num_val_windows = 3

In [1]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
df = pd.read_csv('../../data/preprocessed/base.csv', sep=',')
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [3]:
dfg = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
dfg.head()

,periodo,product_id,tn
0,201701,20001,934.77222
1,201701,20002,550.15707
2,201701,20003,1063.45835
3,201701,20004,555.91614
4,201701,20005,494.27011


In [4]:
dfg['periodo_dt'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'tn': 'target', 'product_id':'item_id', 'periodo_dt': 'timestamp'}, inplace=True)
dfg.drop(columns=['periodo'], inplace=True)

In [5]:
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
productos_ok.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [6]:
dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]

In [7]:
data = TimeSeriesDataFrame(dfg)
data

,,target
item_id,timestamp,
20001,2017-01-01,934.77222
20002,2017-01-01,550.15707
20003,2017-01-01,1063.45835
20004,2017-01-01,555.91614
20005,2017-01-01,494.27011
...,...,...
21263,2019-12-01,0.01270
21265,2019-12-01,0.05007
21266,2019-12-01,0.05121


In [8]:
predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M").fit(data, num_val_windows = 3)
predictions = predictor.predict(data)

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\autogluon\timeseries\predictor.py:197: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  std_freq = pd.tseries.frequencies.to_offset(self.freq).freqstr
Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\model_autogluon\AutogluonModels\ag-20250605_145132'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       5.61 GB / 15.89 GB (35.3%)
Disk Space Avail:   415.98 GB / 893.49 GB (46.6%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 3,
 

In [9]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1189.945568
3,20002,987.819021
5,20003,605.067834
7,20004,466.975746
9,20005,479.402112


In [10]:
predictions_v1.shape

(780, 2)

In [11]:
predictions_v1.to_csv("../../outputs/prediccion_autogluon_3ventanas.csv", sep = ",", index = False)